In [1]:
# Etapa 1: Query SQL para lista epi

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_obras@onelake.dfs.fabric.microsoft.com/lk_obras.Lakehouse/Tables/tab_gold_fato_lista_epi"

tmov = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/tmov"
)
tmov.createOrReplaceTempView("tmov")

titmmov = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/titmmov"
)
titmmov.createOrReplaceTempView("titmmov")

tprd = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/tprd"
)
tprd.createOrReplaceTempView("tprd")

tproduto = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/tproduto"
)
tproduto.createOrReplaceTempView("tproduto")

ttmv = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/ttmv"
)
ttmv.createOrReplaceTempView("ttmv")


df_resultado = spark.sql("""
WITH base AS (
    SELECT
        p.CODCOLIGADA,
        p.CODIGOPRD,
        p.DESCRICAO                                     AS DESCRICAO_PRODUTO,
        p.CODTBORCAMENTO                                AS NATUREZA_ORCAMENTARIA,
        m.NUMEROMOV                                     AS NF,
        m.SERIE,
        m.DATAEMISSAO,
        --Origem efetiva: se TMOV.CODLOC vier nulo, usar TITMMOV.CODLOC
        COALESCE(m.CODLOC, i.CODLOC)                    AS LOCAL_ESTOQUE_ORIGEM,
        m.CODLOCDESTINO                                 AS LOCAL_ESTOQUE_DESTINO,
        m.CODCCUSTO,
        i.PRECOUNITARIO                                 AS PRECO_UNITARIO,
        i.VALORBRUTOITEM                                AS PRECO_TOTAL,
        m.USUARIOCRIACAO                                AS USUARIO,
        m.CODTMV                                        AS MOVIMENTO,
        t.NOME                                          AS DESCRICAO_MOVIMENTO,
        m.CODLOC                                        AS CODLOC_TMOV,     -- mantém para referência
        i.CODLOC                                        AS CODLOC_TITMMOV,  -- mantém para referência
        i.QUANTIDADE
    FROM tmov m
    INNER JOIN titmmov i
        ON i.IDMOV = m.IDMOV
       AND i.CODCOLIGADA = m.CODCOLIGADA
    INNER JOIN tprd p
        ON p.IDPRD = i.IDPRD
       AND p.CODCOLIGADA = i.CODCOLIGADA
    INNER JOIN tproduto tp 
        ON p.CODIGOPRD = tp.CODIGOPRD 
       AND p.CODCOLIGADA = tp.CODCOLPRD  
    INNER JOIN ttmv t
        ON t.CODTMV = m.CODTMV
       AND t.CODCOLIGADA = m.CODCOLIGADA
    WHERE
        m.CODCOLIGADA = 2
        AND m.STATUS <> 'C'
        AND substring(p.CODTBORCAMENTO, 1, 4) = '2.08'
        AND tp.TIPO = 'P'
        AND i.QUANTIDADE <> 0
        AND m.CODTMV IN ('1.1.02', '1.2.01', '1.2.02', '1.2.14', '1.2.15', '1.2.20', '1.2.33', '1.2.70', '3.1.01', '3.1.03', '3.1.04', '4.1.06')
)
SELECT
    CODCOLIGADA,
    CODIGOPRD,
    DESCRICAO_PRODUTO,
    NATUREZA_ORCAMENTARIA,
    NF,
    SERIE,
    DATAEMISSAO,
    LOCAL_ESTOQUE_ORIGEM,
    LOCAL_ESTOQUE_DESTINO,
    CODCCUSTO,
    substring(CODCCUSTO, 9, 5) AS CODCCUSTO_REDUZIDO, 
    PRECO_UNITARIO,
    PRECO_TOTAL,
    USUARIO,
    MOVIMENTO,
    DESCRICAO_MOVIMENTO,
    CASE 
        WHEN (
                (
                    LOCAL_ESTOQUE_ORIGEM = '01201.02'
                    OR substring(LOCAL_ESTOQUE_ORIGEM, length(LOCAL_ESTOQUE_ORIGEM) - 2, 3) = '.17'
                )
                OR LOCAL_ESTOQUE_DESTINO IS NULL
             )
             AND MOVIMENTO <> '4.1.06'
        THEN 'NOVOS'
        ELSE 'USADOS'
    END AS TIPO_MATERIAL,
    QUANTIDADE
FROM base;
""")

# Salva no Lakehouse de destino usando caminho físico
df_resultado.write.format("delta").mode("overwrite").save(path_destino)


StatementMeta(, 700c9e46-5303-4bce-b276-bac32b16ac7a, 3, Finished, Available, Finished)